<a href="https://colab.research.google.com/github/jfahadi/Android-Project/blob/master/federate.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [18]:
import tensorflow as tf
import tensorflow_federated as tff
import numpy as np
# Define the model
def create_rnn_model(input_shape, output_size):
    model = tf.keras.Sequential([
        tf.keras.layers.Input(shape=input_shape),
        tf.keras.layers.SimpleRNN(128, activation='relu', return_sequences=False),
        tf.keras.layers.Dense(output_size, activation='softmax')
    ])
    return model

# Model function for TFF
def model_fn():
    input_spec = {
        'x': tf.TensorSpec(shape=(None, 28, 28), dtype=tf.float32),
        'y': tf.TensorSpec(shape=(None,), dtype=tf.int32),
    }
    keras_model = create_rnn_model(input_shape=(28, 28), output_size=10)
    return tff.learning.models.from_keras_model(
        keras_model=keras_model,
        input_spec=input_spec,
        loss=tf.keras.losses.SparseCategoricalCrossentropy(),
        metrics=[tf.keras.metrics.SparseCategoricalAccuracy()]
    )

# Define optimizer functions
client_optimizer_fn = lambda: tf.keras.optimizers.Adam(learning_rate=0.001)
server_optimizer_fn = lambda: tf.keras.optimizers.SGD(learning_rate=1.0)


# Build the Federated Averaging algorithm
iterative_process = tff.learning.algorithms.build_weighted_fed_avg(
    model_fn=model_fn,
    client_optimizer_fn=client_optimizer_fn,
    server_optimizer_fn=server_optimizer_fn
)

# Initialize the federated process
state = iterative_process.initialize()

# Simulated federated data
federated_data = [
    tf.data.Dataset.from_tensor_slices({
        'x': tf.random.uniform((10, 28, 28)),  # Example input data
        'y': tf.random.uniform((10,), maxval=10, dtype=tf.int32)  # Example labels
    }).batch(2)
]

# Federated training loop
NUM_ROUNDS = 10
for round_num in range(1, NUM_ROUNDS + 1):
    state, metrics = iterative_process.next(state, federated_data)
    print(f"Round {round_num}, Metrics: {metrics}")


Round 1, Metrics: OrderedDict([('distributor', ()), ('client_work', OrderedDict([('train', OrderedDict([('sparse_categorical_accuracy', 0.0), ('loss', 2.3916862), ('num_examples', 10), ('num_batches', 5)]))])), ('aggregator', OrderedDict([('mean_value', ()), ('mean_weight', ())])), ('finalizer', OrderedDict([('update_non_finite', 0)]))])
Round 2, Metrics: OrderedDict([('distributor', ()), ('client_work', OrderedDict([('train', OrderedDict([('sparse_categorical_accuracy', 0.2), ('loss', 2.2291627), ('num_examples', 10), ('num_batches', 5)]))])), ('aggregator', OrderedDict([('mean_value', ()), ('mean_weight', ())])), ('finalizer', OrderedDict([('update_non_finite', 0)]))])
Round 3, Metrics: OrderedDict([('distributor', ()), ('client_work', OrderedDict([('train', OrderedDict([('sparse_categorical_accuracy', 0.2), ('loss', 2.1243775), ('num_examples', 10), ('num_batches', 5)]))])), ('aggregator', OrderedDict([('mean_value', ()), ('mean_weight', ())])), ('finalizer', OrderedDict([('update_n

# Function to Set Weights in Keras Model

In [34]:
def set_model_weights(model, iterative_process, tff_state):
    """
    Load TFF model weights into a Keras model.
    Parameters:
        - model: A Keras model instance.
        - iterative_process: The TFF iterative process.
        - tff_state: LearningAlgorithmState object from TFF.
    """
    # Access the model weights using get_model_weights() on iterative_process
    tff_weights = iterative_process.get_model_weights(tff_state)

    # Assign the TFF weights to the Keras model
    tff_weights.assign_weights_to(model)

# Generate Synthetic Test Data

In [35]:
def generate_synthetic_time_series_data(num_samples, timesteps, num_features, output_size):
    """
    Generate synthetic time-series data for evaluation.
    Parameters:
        - num_samples: Number of samples.
        - timesteps: Number of timesteps per sample.
        - num_features: Number of features per timestep.
        - output_size: Number of output classes (for classification).
    """
    X = np.random.rand(num_samples, timesteps, num_features)
    y = np.random.randint(0, output_size, size=(num_samples,))
    return X, y

# Define input shape and output size
input_shape = (28, 28)  # Input shape matching federated training
output_size = 10        # Number of output classes

# Load the Global Weights into a New Keras Model

In [39]:
rnn_model = create_rnn_model(input_shape=input_shape, output_size=output_size)
set_model_weights(rnn_model, iterative_process, state)

# Generate Synthetic Test Data
X_test, y_test = generate_synthetic_time_series_data(100, input_shape[0], input_shape[1], output_size)





In [43]:
rnn_model = create_rnn_model(input_shape=input_shape, output_size=output_size)
set_model_weights(rnn_model, iterative_process, state)

# Compile the model before evaluation
rnn_model.compile(optimizer='adam', # You can choose any suitable optimizer
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])

# Generate Synthetic Test Data
X_test, y_test = generate_synthetic_time_series_data(100, input_shape[0], input_shape[1], output_size)

# Evaluate Global Model
eval_metrics = rnn_model.evaluate(X_test, y_test, verbose=2)
print("Evaluation Metrics:", eval_metrics)

4/4 - 1s - loss: 2.8768 - accuracy: 0.1300 - 548ms/epoch - 137ms/step
Evaluation Metrics: [2.876816749572754, 0.12999999523162842]
